In [1]:
library(tidyverse)
library(randomForest)


Warning message:
"package 'tidyverse' was built under R version 4.2.3"
Warning message:
"package 'ggplot2' was built under R version 4.2.3"
Warning message:
"package 'tibble' was built under R version 4.2.3"
Warning message:
"package 'tidyr' was built under R version 4.2.3"
Warning message:
"package 'readr' was built under R version 4.2.3"
Warning message:
"package 'purrr' was built under R version 4.2.3"
Warning message:
"package 'dplyr' was built under R version 4.2.3"
Warning message:
"package 'stringr' was built under R version 4.2.3"
Warning message:
"package 'forcats' was built under R version 4.2.3"
Warning message:
"package 'lubridate' was built under R version 4.2.3"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ──────────────────────────────────────

In [7]:
import numpy as np
def balance_logloss(y_true, y_pred):
    y_pred = np.clip([[1,1,1,0,0,0]], 1e-15, 1-1e-15)
    #y_pred / np.sum(y_pred)[:, None]
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    
    logloss = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / (w0+w1)
    
    return logloss

balance_logloss(y_true=np.array([1,0,1,0,1,0]),y_pred = np.array([0.99,0.01,0.99,0.01,0.99,0.01]))

AxisError: axis 1 is out of bounds for array of dimension 1

In [10]:
options(warn = -1)
training_set <- read.csv("D:/7. Kaggle/ICR - Identifying Age-Related Conditions/train.csv", row.names="Id")
test_set <- read.csv("D:/7. Kaggle/ICR - Identifying Age-Related Conditions/test.csv", row.names="Id")
greeks <- read.csv("D:/7. Kaggle/ICR - Identifying Age-Related Conditions/greeks.csv")
submission <- read.csv("D:/7. Kaggle/ICR - Identifying Age-Related Conditions/sample_submission.csv")


In [3]:

na2median <- function(data, col_with_miss){
  sapply(col_with_miss, function(miss_col){
  median_ <- median(na.omit(data[,miss_col]))
  data[,miss_col][is.na(data[,miss_col])] <<- ifelse(is.na(median_),0,median_)
  })
  return(data)

}

transformer <- function(data){
    count_table_ <- table(data)
    freq_table_ <- count_table_/sum(count_table_ )
    freq_ <- freq_table_[match(data, names(freq_table_))]
    return(freq_)
}

transformer_v2 <- function(data, target){
    data_factor <- as.factor(data)
    freq_table <- table(data_factor[target==1])/sum(target==1)
    freq_  <- freq_table[match(data_factor , names(freq_table))]
    return(freq_)
}

seperated_data_with_miss_value <- function(data, col_with_miss, target_col){
    data_clean <- na.omit(data) #擷取沒有NA值的部分
    data_for_train_miss <- data_clean[,!is.element(colnames(data_clean ),c(setdiff(col_with_miss,target_col),"Class"))] #用剩餘的來預測
    target_with_miss <- data[is.na(data[,target_col]), -which(colnames(data)=="Class")]
    return(list("data_for_train_miss"= data_for_train_miss, "target_with_miss"= target_with_miss))

}

predict_missing_value <- function(target_col, col_with_miss, train_data, test_data=NULL){
    print(paste0("Current: ",target_col))
    for_missing_value_pred <- seperated_data_with_miss_value(train_data, col_with_miss, target_col)
    target_with_miss <- for_missing_value_pred$"target_with_miss"
    model_for_target <- randomForest(as.formula(paste0(target_col,"~.")), for_missing_value_pred$"data_for_train_miss")
    print("Modeling Finished")
    for_predicted <-     if(length(test_data)==0){
                                target_with_miss[,-which(colnames(target_with_miss)==target_col)]
                            }else{
                                test_data
                            }

    pred_value <- predict(model_for_target, for_predicted)
    print("Predicting finished")

    names <- rownames(target_with_miss)
    if(length(test_data)==0){
      train_data[is.na(train_data[,target_col]),target_col] <- pred_value 
      return(train_data)
    }else{
      test_data[,target_col] <- pred_value
      return(test_data)
    }
}


data_for_validation <- function(train_size, data){
    sample_index <- sample(1:nrow(data), size= nrow(data) * train_size)
    train_set_ <- data[sample_index ,]
    validat_set_ <- data[-sample_index ,]
    return(list("trainset" = train_set_, "validatset" = validat_set_ , "index" = sample_index  ))
}


balan_logloss <- function(pred_value, true_value){
    pred_value<- sapply(pred_value,function(x) max(min(x, 1-10^-15),10^-15)) #avoid Inf value
    balan_logloss <- (mean(-log(pred_value)*as.numeric(is.element(true_value, 1)))+
    mean(-log(1-pred_value)*as.numeric(!is.element(true_value, 1))))/2
    return(balan_logloss )
 }


string2freq <- function(data, target, mode ){
    if(mode==1){
        renew_variable <- sapply(which(sapply(data, is.character)),function(x) data[,x] <<- as.numeric(transformer(data[,x])))
    }else if(mode==2){
        renew_variable <- sapply(which(sapply(data, is.character)),function(x) data[,x] <<- as.numeric(transformer_v2(data[,x],data[,target])))
    }
  return(data)
 }

greek2freq <- function(data, greeks, mode){
  greeks_matched <- greeks[match(rownames(data), greeks[,"Id"]),]
  if(mode==1){
    freq_greek <- sapply(c("Alpha", "Beta", "Gamma", "Delta"), function(x) transformer(greeks_matched[,x]))
  }else if(mode==2){
    freq_greek <- sapply(c("Alpha", "Beta", "Gamma", "Delta"), function(x) transformer_v2(greeks_matched[,x], data[,"Class"]))
  }

  freq_greek <- as.data.frame(freq_greek)
  rownames(freq_greek) <- greeks_matched[,"Id"]
  return(freq_greek )
}

add_new_feature <- function(data, variables, target, test_data){
  formula_ <- as.formula(paste0(target, " ~ ", paste0(variables, collapse="+")))
  model_for_new_feature <- randomForest(formula_, data)
  pred <- predict(model_for_new_feature, test_data) 
  return(pred)
}

miss_value_recovery<- function(train_data, test_data=NULL){
  if(length(test_data)==0){
    print("Train dataset recovery")
    na_col <- which(apply(train_data, 2, function(x) sum(is.na(x)))>0)
    col_with_miss <- names(na_col) #other columns with missing values
    print(paste0("Col with miss: ",col_with_miss))
    for_notprint <- sapply(col_with_miss, function(x)  train_data <<- predict_missing_value(x, col_with_miss, train_data) )
    return(train_data)
  }else{
    print("Test dataset recovery")
    na_col <- which(apply(test_data, 2, function(x) sum(is.na(x)))>0)
    col_with_miss <- names(na_col) #other columns with missing values
    print(paste0("Col with miss: ",col_with_miss))

    for_notprint <- sapply(col_with_miss, function(x)  test_data <<- predict_missing_value(x, col_with_miss, train_data, test_data) )
    return(test_data)
  }
}


In [8]:
variables <- colnames(training_set)[1:(which(colnames(training_set)=="Class")-1)]
greek_param <- c("greek_alpha","greek_beta", "greek_gamma","greek_delta")

all_roc <- NULL
all_logloss <-NULL

for(i in 1:10){
    print("=====================")
    print(paste0("Run ",i))
    print("=====================")

    training_set$Class <- as.factor(training_set$Class )
    data_for_validation_ <- data_for_validation(0.7, training_set)
    CV_train <- data_for_validation_$trainset 
    CV_validation <- data_for_validation_$validatset 
    CV_train_v2 <- string2freq(CV_train, "Class", mode=1)
    freq_greek_train <- greek2freq(CV_train_v2, greeks, mode=2)

    CV_validation_v2 <- string2freq(CV_validation, "Class", mode=1)
    greek_param <- c("greek_alpha","greek_beta", "greek_gamma","greek_delta")

    CV_train_with_repair <- miss_value_recovery(CV_train_v2,test_data=NULL)
    CV_validation_with_repair <- miss_value_recovery(train_data=CV_validation_v2, test_data=CV_validation_v2)
    train_with_greek <- as.data.frame(cbind(CV_train_with_repair, freq_greek_train )) 
    train_with_greek$greek_alpha  <- add_new_feature(train_with_greek, variables, "Alpha", train_with_greek )
    train_with_greek$greek_beta  <- add_new_feature(train_with_greek, variables, "Beta", train_with_greek )
    train_with_greek$greek_gamma  <- add_new_feature(train_with_greek, variables, "Gamma", train_with_greek )
    train_with_greek$greek_delta  <- add_new_feature(train_with_greek, variables, "Delta", train_with_greek )

    CV_validation_with_repair$greek_alpha  <- add_new_feature(train_with_greek, variables, "greek_alpha" , CV_validation_with_repair )
    CV_validation_with_repair$greek_beta    <- add_new_feature(train_with_greek, variables, "greek_beta" , CV_validation_with_repair)
    CV_validation_with_repair$greek_gamma  <- add_new_feature(train_with_greek, variables, "greek_gamma", CV_validation_with_repair)
    CV_validation_with_repair$greek_delta  <- add_new_feature(train_with_greek, variables, "greek_delta", CV_validation_with_repair)
    #tuneRF_ <-tuneRF(x = CV_train_with_repair[,-57], y = CV_train_with_repair[,57], ntreeTry=300, mtryStart = 1)
    #mtry_ <- tuneRF_[which.min(tuneRF_[,"OOBError"]),"mtry"]
    #model_RF <- randomForest(as.formula(paste0("Class~", paste0(c(variables,greek_param), collapse="+"))), train_with_greek, ntree = 300, mtry = mtry_)
    model_RF <- randomForest(as.formula(paste0("Class ~", paste0(c(variables, greek_param), collapse="+"))), train_with_greek)
    pred_RF_validation <- predict(model_RF , CV_validation_with_repair, type="prob")
    logloss<- balan_logloss(as.numeric(pred_RF_validation[,2]), as.numeric(as.character(CV_validation_with_repair$Class)))
    print(logloss)
    all_logloss  <- c(all_logloss, logloss) 
  }

View(as.data.frame(all_logloss ))

[1] "====================="
[1] "Run 1"
[1] "====================="
[1] "Train dataset recovery"
[1] "Col with miss: BQ" "Col with miss: CB" "Col with miss: CC"
[4] "Col with miss: DU" "Col with miss: EL" "Col with miss: FC"
[7] "Col with miss: FL" "Col with miss: FS" "Col with miss: GL"
[1] "Current: BQ"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: CB"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: CC"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: DU"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: EL"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: FC"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: FL"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: FS"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: GL"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Test dataset recovery"
[1] "Col with miss: BQ" "Col with miss: EL"
[1] "Current: BQ"
[1] "Mode

all_logloss
<dbl>
0.1558675
0.1339169
0.1277500
0.1342982
0.1189915
0.1021413
0.1126644
0.1726152
0.1696413


In [11]:
training_set$Class <- as.factor(training_set$Class )

CV_train <- training_set
CV_train_v2 <- string2freq(CV_train, "Class", mode=1)
freq_greek_train <- greek2freq(CV_train_v2, greeks, mode=1)

CV_validation <- test_set 
CV_validation_v2 <- string2freq(CV_validation, "Class",mode=1)

variables <- colnames(training_set)[1:(which(colnames(training_set)=="Class")-1)]
greek_param <- c("greek_alpha","greek_beta", "greek_gamma","greek_delta")

CV_train_with_repair <- miss_value_recovery(CV_train_v2,test_data=NULL)
CV_validation_with_repair <- miss_value_recovery(train_data = CV_validation_v2, test_data = CV_validation_v2)
train_with_greek <- as.data.frame(cbind(CV_train_with_repair, freq_greek_train )) 


[1] "Train dataset recovery"
[1] "Col with miss: BQ" "Col with miss: CB" "Col with miss: CC"
[4] "Col with miss: DU" "Col with miss: EL" "Col with miss: FC"
[7] "Col with miss: FL" "Col with miss: FS" "Col with miss: GL"
[1] "Current: BQ"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: CB"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: CC"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: DU"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: EL"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: FC"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: FL"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: FS"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Current: GL"
[1] "Modeling Finished"
[1] "Predicting finished"
[1] "Test dataset recovery"
[1] "Col with miss: "


In [12]:
train_with_greek$greek_alpha  <- add_new_feature(train_with_greek, variables, "Alpha", train_with_greek )
train_with_greek$greek_beta  <- add_new_feature(train_with_greek, variables, "Beta", train_with_greek )
train_with_greek$greek_gamma  <- add_new_feature(train_with_greek, variables, "Gamma", train_with_greek )
train_with_greek$greek_delta  <- add_new_feature(train_with_greek, variables, "Delta", train_with_greek )

CV_validation_with_repair$greek_alpha  <- add_new_feature(train_with_greek, variables, "greek_alpha" , CV_validation_with_repair )
CV_validation_with_repair$greek_beta    <- add_new_feature(train_with_greek, variables, "greek_beta" , CV_validation_with_repair)
CV_validation_with_repair$greek_gamma  <- add_new_feature(train_with_greek, variables, "greek_gamma", CV_validation_with_repair)
CV_validation_with_repair$greek_delta  <- add_new_feature(train_with_greek, variables, "greek_delta", CV_validation_with_repair)

model_RF <- randomForest(as.formula(paste0("Class ~", paste0(c(variables, greek_param), collapse="+"))), train_with_greek)
pred_RF_test <- predict(model_RF , CV_validation_with_repair, type="prob")
pred_RF_test <- as.data.frame(pred_RF_test, row.names = NULL)

submission <- data.frame(cbind(as.character(row.names(pred_RF_test )), pred_RF_test),row.names=NULL)
colnames(submission ) <- c("Id", "class_0", "class_1")
write_delim(submission, file="submission.csv", delim=',')
print(submission)

            Id class_0 class_1
1 00eed32682bb   0.468   0.532
2 010ebe33f668   0.468   0.532
3 02fa521e1838   0.468   0.532
4 040e15f562a2   0.468   0.532
5 046e85c7cc7f   0.468   0.532
